In [5]:
import httpx
import asyncio
import datetime as dt
import pylint
from lxml import etree
import time

In [84]:
OLDEST = dt.datetime(1958, 8, 4)

In [85]:
def round_date(date, scalar=1):
    # rounds the date to a Saturday
    while date.weekday() != 5:
        date = date + scalar*dt.timedelta(days=1)
    return date

In [86]:
def date_generator(init_d, delta=1):
    # infintite generator for dates that takes the timedelta as a paramater
    init_d = round_date(init_d)
    curr = init_d
    while True:
        yield curr
        curr -= dt.timedelta(weeks=delta)

In [ ]:
def search_chunk(html_chunk):
    # testing xpaths
    html_text = httpx.get('https://www.billboard.com/charts/hot-100/1962-07-21/').text
    tree = html.document_fromstring(html_text, parser)
    container_path = "//html//body//main//div[@class='o-chart-results-list-row-container']/ul"
    for row in tree.xpath(container_path):
        print(row.xpath("./li/span/text()")[0].strip())
        print(row.xpath("./li//h3/text()")[0].strip())
        print(row.xpath(".//li/span[not(contains(text(), 'NEW'))]/text()")[1].strip())
        print(row.xpath(".//li/span[not(contains(text(), 'NEW'))]/text()")[4].strip())



1
Roses Are Red (My Love)
Bobby Vinton
7
2
The Wah Watusi
The Orlons
7
3
I Can't Stop Loving You
Ray Charles
12
4
The Stripper
David Rose and His Orchestra
11
5
Sealed With A Kiss
Brian Hyland
7
6
Wolverton Mountain
Claude King
9
7
Johnny Get Angry
Joanie Sommers
9
8
Speedy Gonzales
Pat Boone
6
9
Gravy (For My Mashed Potatoes)
Dee Dee Sharp
6
10
Palisades Park
Freddy Cannon
11
11
Al Di La'
Emilio Pericoli
10
12
Ahab, The Arab
Ray Stevens
4
13
Breaking Up Is Hard To Do
Neil Sedaka
4
14
It Keeps Right On A-Hurtin'
Johnny Tillotson
11
15
Dancin' Party
Chubby Checker
5
16
I'll Never Dance Again
Bobby Rydell
8
17
Having A Party
Sam Cooke
9
18
Theme From Dr. Kildare (Three Stars Will Shine Tonight)
Richard Chamberlain
8
19
(Girls, Girls, Girls) Made To Love
Eddie Hodges
5
20
Snap Your Fingers
Joe Henderson
10
21
Johnny Loves Me
Shelley Fabares
7
22
Twist And Shout
The Isley Brothers
8
23
You'll Lose A Good Thing
Barbara Lynn
6
24
I Need Your Loving
Don Gardner And Dee Dee Ford
8
25
Playboy
T

In [ ]:
# testing parser feed() and read_events() methods
async def feed_parser():
    parser = etree.HTMLPullParser(events=('end',), tag='ul')
    async with client.stream('GET', URL) as response:
        async for chunk in response.aiter_text():
            parser.feed(chunk)
            for event, ele in parser.read_events():
                search_xpath(ele)
                

Help on AsyncClient in module httpx object:

class AsyncClient(httpx._client.BaseClient)
 |  AsyncClient(
 |      *,
 |      auth: 'AuthTypes | None' = None,
 |      params: 'QueryParamTypes | None' = None,
 |      headers: 'HeaderTypes | None' = None,
 |      cookies: 'CookieTypes | None' = None,
 |      verify: 'ssl.SSLContext | str | bool' = True,
 |      cert: 'CertTypes | None' = None,
 |      http1: 'bool' = True,
 |      http2: 'bool' = False,
 |      proxy: 'ProxyTypes | None' = None,
 |      mounts: 'None | typing.Mapping[str, AsyncBaseTransport | None]' = None,
 |      timeout: 'TimeoutTypes' = Timeout(timeout=5.0),
 |      follow_redirects: 'bool' = False,
 |      limits: 'Limits' = Limits(max_connections=100, max_keepalive_connections=20, keepalive_expiry=5.0),
 |      max_redirects: 'int' = 20,
 |      event_hooks: 'None | typing.Mapping[str, list[EventHook]]' = None,
 |      base_url: 'URL | str' = '',
 |      transport: 'AsyncBaseTransport | None' = None,
 |      trust_e

In [46]:
date = round_date(dt.datetime.strptime('1965-05-23','%Y-%m-%d'))
with httpx.Client(base_url="https://www.billboard.com/charts/hot-100/",timeout=10.0) as client:
    e = catch_404(client) 

the date 1958-03-01 00:00:00 isn't equal to 1958-08-04 00:00:00, interval 260 chaning at 0.5434670448303223 seconds
the date 1961-02-25 00:00:00 isn't equal to 1961-02-27 00:00:00, interval 52 chaning at 1.150543212890625 seconds
the date 1961-08-26 00:00:00 isn't equal to 1961-08-28 00:00:00, interval 26 chaning at 1.5880730152130127 seconds
the date 1961-12-02 00:00:00 isn't equal to 1961-12-04 00:00:00, interval 12 chaning at 2.025895118713379 seconds
the date 1961-12-30 00:00:00 isn't equal to 1962-01-06 00:00:00, interval 4 chaning at 2.5752062797546387 seconds
the date 1961-12-30 00:00:00 isn't equal to 1962-01-06 00:00:00, interval 1 chaning at 3.4651572704315186 seconds


In [33]:
e

datetime.datetime(1965, 5, 29, 0, 0)